# Accessing the NHTSA API (crawler)
This notebook will focus on ratings data from NHTSA API.
## Rating Data 

In [27]:
import requests
from pathlib import Path
import json
import pandas as pd
import urllib.request
import urllib.error
import time
from loguru import logger
import pygame
from IPython.display import display, clear_output

In [28]:
# Set API prefix because our api headers are same.
api_prefix = "https://api.nhtsa.gov"

# Set API suffix for get year.
api_get_year = "/SafetyRatings"

# Set API suffix for get SafetyRatings by VehicleId.
api_get_rating = "/SafetyRatings/VehicleId"

# Set output path.
dir_raw = Path("../Raw_Data/Raw_API/Ratings")
dir_raw.mkdir(parents=True, exist_ok=True)
out_dir = Path("../Raw_Data/Raw_API/Ratings_buffer")
out_dir.mkdir(parents=True, exist_ok=True)

In [29]:
def fetch_Ratings(url):
    logger.info("Fetching %s" % url)
    try:
        headers = {
          'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1"
        }
        response = requests.get(url,headers=headers)
        status = response.status_code
        logger.info(status)

        # Stop if the API cannot be accessed properly.
        if status != 200:
            return [-1,None]
        
        # Otherwise return retrieved data.
        raw_data = response.text
        return [1,json.loads(raw_data)]
    except urllib.error.URLError as e:
        e.reason 

# Output each Rating for a json file.
def write_out_raw(Ratings_count,year,make,model,data):
    # Check if '/' is already in make or model, replaced by '_or_'.
    if '/' in make:
        make = make.replace("/", "_or_")
    if '/' in model:
        model = model.replace("/", "_or_")
    # File name format [complaint_num]year_make_model.json
    #             e.g. [802]1994_MERCEDES BENZ_E.json
    fname = "[%d]%s_%s_%s.json" % (Ratings_count,year,make,model)
    out_path = dir_raw / fname
    logger.info("Writing data to %s" % out_path)
    fout = open(out_path, "w")
    json.dump(data, fout, indent=4, sort_keys=True)
    fout.close()
    
# Check API status.
def check_status(data):
    #number of Rating in this time.
    Rating_num = data["count"]
    # Are results returned successfully?
    if data["message"] != "Results returned successfully":
        logger.warning("Fail to get from API, %s" % (data["status_message"]))
        return [False,-1]
    elif Rating_num <= 0:
        logger.warning("No info from current API address.")
        return [False,0]
    else:
        return [True,Rating_num]

# Output for "years", "makes", "models" which are dictionaries we get from first 3 steps.  
def write_out(name,data):
    fname = "%s.json" % (name)
    out_path = out_dir / fname
    logger.info("Writing data to %s" % out_path)
    fout = open(out_path, "w")
    json.dump(data, fout, indent=4, sort_keys=True)
    fout.close()

## Step-1: Get all Model Years 

In [96]:
# Construct API url.
api = api_prefix + api_get_year
# Fetch data.
status,data = fetch_Ratings(api)
# Convert result data into list format.
years = (list(each["ModelYear"] for each in data["Results"]))
logger.info("Completed the task of getting all model years!")
# Store years data.
write_out("Rating-years",years)

2022-04-07 16:05:07.210 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings
2022-04-07 16:05:07.430 | INFO     | __main__:fetch_Ratings:9 - 200
2022-04-07 16:05:07.431 | INFO     | __main__:<module>:7 - Completed the task of getting all model years!
2022-04-07 16:05:07.432 | INFO     | __main__:write_out:55 - Writing data to ../raw/Ratings_buffer/Rating-years.json


## Step-2: Get all Makes for the Model Year

In [120]:
makes={}
for year in years:
    clear_output()
    # Construct API url.
    api = api_prefix + "/SafetyRatings/modelyear/%d" % (int(year))
    # Fetch data.
    status,data = fetch_Ratings(api)
    # Store result data into a dict by year.
    for each in data["Results"]:
        if year in makes:
            makes[year].append(each["Make"])
        else:
            makes[year] = [each["Make"]]
logger.info("Completed the task of getting all makes for the model year!")
# Store makes data.
write_out("Rating_makes",makes)

2022-04-07 16:21:22.343 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/1990
2022-04-07 16:21:22.577 | INFO     | __main__:fetch_Ratings:9 - 200
2022-04-07 16:21:22.578 | INFO     | __main__:<module>:14 - Completed the task of getting all makes for the model year!
2022-04-07 16:21:22.580 | INFO     | __main__:write_out:55 - Writing data to ../raw/Ratings_buffer/Rating_makes.json


## Step-3: Get all Models for the Make and Model Year 
Due to the website's restrictions on crawlers, we have to change IP frequently to bypass its restrictions. The following function will automatically stop when it received an access error (Typically a 403 error) and return the data we got so far. And it will give us a hint for next start.

In [201]:
def Get_model(years,skip):
    models={}
    start_year = years[0]
    for year in years:
        for make in makes[year]:
            # If meet some special case we need skip it.
            if make != skip:
                clear_output()
                api = api_prefix + "/SafetyRatings/modelyear/%d/make/%s" % (int(year),make)
                status,data = fetch_Ratings(api)
            # Status equal -1 means we are failing to use this API at this moment.
            if status == -1:
                logger.warning("Start year:%s, Current Year:%s(index:%d) Current Make:%s(index:%d)" % (start_year,year,years.index(year),make,makes[year].index(make)))
                return models
            # Otherwise, store result data into a dict by make.
            else:
                for each in data["Results"]:
                    if make in models:
                        models[make].append(each["Model"])
                    else:
                        models[make] = [each["Model"]]
    end_year = year
    logger.info("Finished (%s-%s)" % (start_year,end_year))
    return models

In [202]:
models = Get_model(years,"")

2022-04-07 17:24:44.654 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/1990/make/VOLKSWAGEN
2022-04-07 17:24:44.856 | INFO     | __main__:fetch_Ratings:9 - 200
2022-04-07 17:24:44.858 | INFO     | __main__:Get_model:23 - Finished (2023-1990)


#### Delete pair if the value of this key is empty.

In [211]:
models_clean = models.copy()
for key in list(models_clean.keys()):
    if not models_clean.get(key):
        del models_clean[key]
len(models_clean)

60

In [212]:
for k,v in models_clean.items():
    models_clean[k] = list(set(models_clean[k]))
# Keys Sizes & Values Sizes.
len(models_clean["JAGUAR"]),sum([len(x) for x in models_clean.values()])

(25, 2010)

In [213]:
# Handel some special case (name contain '/').
for k,v in models_clean.items():
    for mod in v:
        if '/' in mod:
            # Find position in that dict -> values -> list
            mod_id = v.index(mod)
            # Replace '/' with ' '(space)
            v[mod_id] = mod.replace("/", " ")

In [215]:
write_out("Rating_models",models_clean)

2022-04-07 17:32:38.694 | INFO     | __main__:write_out:55 - Writing data to ../raw/Ratings_buffer/Rating_models.json


## Step-4: Get all Ratings for the selected Model Year, Make, Model by VehicleId

In [216]:
def Get_Ratings(Ratings_count,years,makes,models,continue_):
    Ratings_count = Ratings_count
    start_year = years[0]
    for year in years:
        # "continue_" means we've suffered an interruption, 
        # and we're going back to that position.
        if continue_ : 
            #Get user input of "make" which want to continue.
            make_input = input()
            
            #Find correspond index position in the "makes" dict.
            position_make = makes[year].index(make_input)
            
            #Get user input of "model" which want to continue.
            model_input = input()
            
            #Find correspond index position in the "models" dict.
            position_model = models[make_input].index(model_input)
            
            # Find a list of rest of makes by current year. 
            make_position = makes[year].index(make_input)
            
            # Find a list of rest of models by current year & make. 
            model_position = models[make_input].index(model_input)

            for make in (makes[year])[make_position:]:
                for model in (models[make_input])[model_position:]:
                    
                    clear_output()
                    api = api_prefix + "/SafetyRatings/modelyear/%d/make/%s/model/%s" % (int(year),make,model)
                    # Fetch data.
                    status, data = fetch_Ratings(api)

                    # Status equal -1 means we are failing to use this API at this moment.
                    if result_status(status,Ratings_count,year,make,model) == -1:
                        return -1
                    
                    # Skip if data is empty
                    if data["Results"] != []:
                        for result in data["Results"]:
                            # Get Vehicle ID from current fetched data.
                            vid = result["VehicleId"]
                            # Use id for fetch new rating data.
                            rating_data_status, rating_data = fetch_Ratings(api_prefix+api_get_rating+"/%d"%(int(vid)))
                            # Status equal -1 means we are failing to use this API at this moment.
                            if result_status(rating_data_status,Ratings_count,year,make,model) == -1:
                                return -1
                            #Write out
                            write_out_raw(Ratings_count,year,make,model,rating_data["Results"])
                            Ratings_count+=1
                    else:
                        logger.info("Empty data")
            continue_ = False
        else:
            for make in makes[year]:
                for model in models[make]:
                    clear_output()
                    api = api_prefix + "/SafetyRatings/modelyear/%d/make/%s/model/%s" % (int(year),make,model)
                    status, data = fetch_Ratings(api)

                    # Status equal -1 means we are failing to use this API at this moment.
                    if result_status(status,Ratings_count,year,make,model) == -1:
                        return -1
                    
                    # Skip if data is empty
                    if data["Results"] != []:
                        for result in data["Results"]:
                            # Get Vehicle ID from current fetched data.
                            vid = result["VehicleId"]
                            # Use id for fetch new rating data.
                            rating_data_status, rating_data = fetch_Ratings(api_prefix+api_get_rating+"/%d"%(int(vid)))
                            # Status equal -1 means we are failing to use this API at this moment.
                            if result_status(rating_data_status,Ratings_count,year,make,model) == -1:
                                return -1
                            #Write out
                            write_out_raw(Ratings_count,year,make,model,rating_data["Results"])
                            Ratings_count+=1
                    else:
                        logger.info("Empty data")
    end_year = year
    logger.info("Finished (%s_%s_%s_%s) from %s to %s." % (Ratings_count,year,make,model,start_year,end_year))

def result_status(status,Ratings_count,year,make,model):
    if status == -1:
        logger.warning("Stopped! (%s_%s_%s_%s)" % (Ratings_count,year,make,model)) 
        alarm_clock()
        return -1
    return 0

# Set an alert to remind me to reset when our IP is blocked.
def alarm_clock():
    file = 'alarm_music.mp3'
    pygame.init()
    pygame.mixer.init()
    pygame.mixer.music.load(file)
    pygame.mixer.music.play()
    time.sleep(10)
    pygame.mixer.music.stop()

In [ ]:
Get_Ratings(0,years,makes,models_clean,False)

In [222]:
start = years.index(2022)
Get_Ratings(778,years[start:],makes,models_clean,True)

2022-04-07 19:08:00.757 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2020/make/MAZDA/model/RX-8
2022-04-07 19:08:01.002 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-07 19:08:01.003 | WARNING  | __main__:result_status:85 - Stopped! (2148_2020_MAZDA_RX-8)


-1

In [223]:
start = years.index(2020)
Get_Ratings(2148,years[start:],makes,models_clean,True)

2022-04-07 19:40:10.624 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2018/make/AUDI/model/A5 CABRIOLET
2022-04-07 19:40:10.874 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-07 19:40:10.874 | WARNING  | __main__:result_status:85 - Stopped! (3052_2018_AUDI_A5 CABRIOLET)


-1

In [225]:
start = years.index(2018)
Get_Ratings(3052,years[start:],makes,models_clean,True)

2022-04-07 19:58:49.474 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2018/make/MAZDA/model/RS 7
2022-04-07 19:58:49.707 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-07 19:58:49.709 | WARNING  | __main__:result_status:85 - Stopped! (3064_2018_MAZDA_RS 7)


-1

In [228]:
start = years.index(2018)
Get_Ratings(3064,years[start:],makes,models_clean,True)

2022-04-07 20:19:07.833 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2017/make/HYUNDAI/model/ELANTRA GT
2022-04-07 20:19:08.016 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-07 20:19:08.017 | WARNING  | __main__:result_status:85 - Stopped! (3439_2017_HYUNDAI_ELANTRA GT)


-1

In [230]:
start = years.index(2017)
Get_Ratings(3439,years[start:],makes,models_clean,True)

2022-04-07 20:33:59.919 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2016/make/BMW/model/X3 XDRIVE28I
2022-04-07 20:34:00.521 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-07 20:34:00.522 | WARNING  | __main__:result_status:85 - Stopped! (3551_2016_BMW_X3 XDRIVE28I)


-1

In [231]:
start = years.index(2016)
Get_Ratings(3551,years[start:],makes,models_clean,True)

2022-04-07 20:44:05.362 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2015/make/INFINITI/model/Q45
2022-04-07 20:44:05.759 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-07 20:44:05.760 | WARNING  | __main__:result_status:85 - Stopped! (3965_2015_INFINITI_Q45)


-1

In [232]:
start = years.index(2015)
Get_Ratings(3965,years[start:],makes,models_clean,True)

2022-04-07 20:59:09.629 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2014/make/FORD/model/RANGER
2022-04-07 20:59:09.744 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-07 20:59:09.745 | WARNING  | __main__:result_status:85 - Stopped! (4231_2014_FORD_RANGER)


-1

In [233]:
start = years.index(2014)
Get_Ratings(4231,years[start:],makes,models_clean,True)

2022-04-07 21:08:47.600 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2013/make/GMC/model/SAVANA
2022-04-07 21:08:48.008 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-07 21:08:48.009 | WARNING  | __main__:result_status:85 - Stopped! (4528_2013_GMC_SAVANA)


-1

In [234]:
start = years.index(2013)
Get_Ratings(4528,years[start:],makes,models_clean,True)

2022-04-07 21:19:09.837 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2012/make/BENTLEY/model/CONTINENTAL SUPERSPORTS
2022-04-07 21:19:10.124 | INFO     | __main__:fetch_Ratings:9 - 200
2022-04-07 21:19:10.125 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/VehicleId/6339
2022-04-07 21:19:10.394 | INFO     | __main__:fetch_Ratings:9 - 200
2022-04-07 21:19:10.395 | INFO     | __main__:write_out_raw:32 - Writing data to ../raw/Ratings/[4608]2012_BENTLEY_CONTINENTAL SUPERSPORTS.json
2022-04-07 21:19:10.396 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/VehicleId/6338
2022-04-07 21:19:10.589 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-07 21:19:10.590 | WARNING  | __main__:result_status:85 - Stopped! (4609_2012_BENTLEY_CONTINENTAL SUPERSPORTS)


-1

In [235]:
start = years.index(2012)
Get_Ratings(4609,years[start:],makes,models_clean,True)

2022-04-07 21:29:15.360 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2011/make/HYUNDAI/model/SCOUPE
2022-04-07 21:29:15.540 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-07 21:29:15.541 | WARNING  | __main__:result_status:85 - Stopped! (4942_2011_HYUNDAI_SCOUPE)


-1

In [236]:
start = years.index(2011)
Get_Ratings(4942,years[start:],makes,models_clean,True)

2022-04-07 21:45:09.960 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2010/make/MERCEDES-BENZ/model/CLK-CLASS
2022-04-07 21:45:10.204 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-07 21:45:10.205 | WARNING  | __main__:result_status:85 - Stopped! (5229_2010_MERCEDES-BENZ_CLK-CLASS)


-1

In [237]:
start = years.index(2010)
Get_Ratings(5229,years[start:],makes,models_clean,True)

2022-04-07 21:54:06.958 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2009/make/BMW/model/Z4 SDRIVE35IS
2022-04-07 21:54:07.138 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-07 21:54:07.138 | WARNING  | __main__:result_status:85 - Stopped! (5269_2009_BMW_Z4 SDRIVE35IS)


-1

In [238]:
start = years.index(2009)
Get_Ratings(5269,years[start:],makes,models_clean,True)

2022-04-07 22:34:03.106 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2008/make/JEEP/model/GRAND CHEROKEE WK
2022-04-07 22:34:03.275 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-07 22:34:03.276 | WARNING  | __main__:result_status:85 - Stopped! (5505_2008_JEEP_GRAND CHEROKEE WK)


-1

In [239]:
start = years.index(2008)
Get_Ratings(5505,years[start:],makes,models_clean,True)

2022-04-07 22:44:21.969 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2007/make/KIA/model/RONDO
2022-04-07 22:44:22.378 | INFO     | __main__:fetch_Ratings:9 - 200
2022-04-07 22:44:22.378 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/VehicleId/2337
2022-04-07 22:44:22.570 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-07 22:44:22.571 | WARNING  | __main__:result_status:85 - Stopped! (5710_2007_KIA_RONDO)


-1

In [240]:
start = years.index(2007)
Get_Ratings(5710,years[start:],makes,models_clean,True)

2022-04-07 22:53:58.724 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2006/make/FORD/model/EXPEDITION
2022-04-07 22:53:58.896 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-07 22:53:58.897 | WARNING  | __main__:result_status:85 - Stopped! (5822_2006_FORD_EXPEDITION)


-1

In [241]:
start = years.index(2006)
Get_Ratings(5822,years[start:],makes,models_clean,True)

2022-04-07 23:03:43.294 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2006/make/SAAB/model/FOCUS RS
2022-04-07 23:03:43.495 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-07 23:03:43.496 | WARNING  | __main__:result_status:85 - Stopped! (5836_2006_SAAB_FOCUS RS)


-1

In [244]:
start = years.index(2006)
Get_Ratings(5836,years[start:],makes,models_clean,True)

2022-04-07 23:14:00.980 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2005/make/MERCURY/model/GRAND MARQUIS
2022-04-07 23:14:01.159 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-07 23:14:01.160 | WARNING  | __main__:result_status:85 - Stopped! (6071_2005_MERCURY_GRAND MARQUIS)


-1

In [245]:
start = years.index(2005)
Get_Ratings(6071,years[start:],makes,models_clean,True)

2022-04-07 23:23:58.713 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2004/make/LEXUS/model/SC430
2022-04-07 23:23:58.988 | INFO     | __main__:fetch_Ratings:9 - 200
2022-04-07 23:23:58.989 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/VehicleId/1323
2022-04-07 23:23:59.159 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-07 23:23:59.160 | WARNING  | __main__:result_status:85 - Stopped! (6269_2004_LEXUS_SC430)


-1

In [248]:
start = years.index(2004)
Get_Ratings(6269,years[start:],makes,models_clean,True)

2022-04-08 08:23:51.187 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2003/make/DODGE/model/GRAND CARAVAN CARGO
2022-04-08 08:23:51.221 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-08 08:23:51.223 | WARNING  | __main__:result_status:85 - Stopped! (6364_2003_DODGE_GRAND CARAVAN CARGO)


-1

In [249]:
start = years.index(2003)
Get_Ratings(6364,years[start:],makes,models_clean,True)

2022-04-08 08:48:54.415 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2002/make/HYUNDAI/model/TUCSON PLUG-IN HYBRID
2022-04-08 08:48:54.585 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-08 08:48:54.585 | WARNING  | __main__:result_status:85 - Stopped! (6522_2002_HYUNDAI_TUCSON PLUG-IN HYBRID)


-1

In [255]:
start = years.index(2001)
Get_Ratings(6556,years[start:],makes,models_clean,True)

2022-04-08 09:28:49.741 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/2000/make/PONTIAC/model/G5
2022-04-08 09:28:49.914 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-08 09:28:49.915 | WARNING  | __main__:result_status:85 - Stopped! (6684_2000_PONTIAC_G5)


-1

In [264]:
start = years.index(1999)
Get_Ratings(6707,years[start:],makes,models_clean,True)

2022-04-08 09:41:17.129 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/1999/make/NISSAN/model/TYPHOON
2022-04-08 09:41:17.492 | INFO     | __main__:fetch_Ratings:9 - 502
2022-04-08 09:41:17.493 | WARNING  | __main__:result_status:85 - Stopped! (6712_1999_NISSAN_TYPHOON)


-1

In [269]:
start = years.index(1999)
Get_Ratings(6712,years[start:],makes,models_clean,True)

2022-04-08 09:53:50.689 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/1998/make/VOLVO/model/XC90 T8
2022-04-08 09:53:50.873 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-08 09:53:50.875 | WARNING  | __main__:result_status:85 - Stopped! (6824_1998_VOLVO_XC90 T8)


-1

In [270]:
start = years.index(1998)
Get_Ratings(6824,years[start:],makes,models_clean,True)

2022-04-08 10:03:53.247 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/1996/make/BMW/model/M6
2022-04-08 10:03:53.416 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-08 10:03:53.418 | WARNING  | __main__:result_status:85 - Stopped! (6981_1996_BMW_M6)


-1

In [271]:
start = years.index(1996)
Get_Ratings(6981,years[start:],makes,models_clean,True)

2022-04-08 10:13:45.134 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/1996/make/MAZDA/model/2 SERIES COUPE
2022-04-08 10:13:45.308 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-08 10:13:45.310 | WARNING  | __main__:result_status:85 - Stopped! (6981_1996_MAZDA_2 SERIES COUPE)


-1

In [279]:
start = years.index(1996)
Get_Ratings(6980,years[start:],makes,models_clean,True)

2022-04-08 10:23:49.706 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/1996/make/GMC/model/X5 SDRIVE35I
2022-04-08 10:23:49.881 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-08 10:23:49.882 | WARNING  | __main__:result_status:85 - Stopped! (6981_1996_GMC_X5 SDRIVE35I)


-1

In [282]:
start = years.index(1996)
Get_Ratings(6981,years[start:],makes,models_clean,True)

2022-04-08 10:33:55.338 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/1995/make/AUDI/model/AUDI S8 LWB
2022-04-08 10:33:55.525 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-08 10:33:55.527 | WARNING  | __main__:result_status:85 - Stopped! (6988_1995_AUDI_AUDI S8 LWB)


-1

In [283]:
start = years.index(1995)
Get_Ratings(6988,years[start:],makes,models_clean,True)

2022-04-08 10:43:54.956 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/1995/make/OLDSMOBILE/model/A5 SPORTBACK
2022-04-08 10:43:55.132 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-08 10:43:55.134 | WARNING  | __main__:result_status:85 - Stopped! (6989_1995_OLDSMOBILE_A5 SPORTBACK)


-1

In [286]:
start = years.index(1995)
Get_Ratings(6989,years[start:],makes,models_clean,True)

2022-04-08 10:53:51.140 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/1993/make/CHEVROLET/model/MALIBU
2022-04-08 10:53:51.399 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-08 10:53:51.400 | WARNING  | __main__:result_status:85 - Stopped! (7134_1993_CHEVROLET_MALIBU)


-1

In [287]:
start = years.index(1993)
Get_Ratings(7134,years[start:],makes,models_clean,True)

2022-04-08 11:03:54.689 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/1991/make/CHEVROLET/model/CAPRICE PPV
2022-04-08 11:03:54.861 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-08 11:03:54.862 | WARNING  | __main__:result_status:85 - Stopped! (7251_1991_CHEVROLET_CAPRICE PPV)


-1

In [288]:
start = years.index(1991)
Get_Ratings(7251,years[start:],makes,models_clean,True)

2022-04-08 11:13:54.351 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/1991/make/MAZDA/model/HHR
2022-04-08 11:13:54.535 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-08 11:13:54.537 | WARNING  | __main__:result_status:85 - Stopped! (7261_1991_MAZDA_HHR)


-1

In [291]:
start = years.index(1991)
Get_Ratings(7261,years[start:],makes,models_clean,True)

2022-04-08 11:23:30.264 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/1990/make/INFINITI/model/Q70 HYBRID
2022-04-08 11:23:30.658 | INFO     | __main__:fetch_Ratings:9 - 403
2022-04-08 11:23:30.659 | WARNING  | __main__:result_status:85 - Stopped! (7294_1990_INFINITI_Q70 HYBRID)


-1

In [292]:
start = years.index(1990)
Get_Ratings(7294,years[start:],makes,models_clean,True)

2022-04-08 11:29:20.183 | INFO     | __main__:fetch_Ratings:2 - Fetching https://api.nhtsa.gov/SafetyRatings/modelyear/1990/make/VOLKSWAGEN/model/G35 SEDAN
2022-04-08 11:29:20.473 | INFO     | __main__:fetch_Ratings:9 - 200
2022-04-08 11:29:20.474 | INFO     | __main__:Get_Ratings:52 - Empty data
2022-04-08 11:29:20.475 | INFO     | __main__:Get_Ratings:81 - Finished (7295_1990_VOLKSWAGEN_G35 SEDAN) from 1990 to 1990.
